In [2]:
import pandas as pd
from repsim.benchmark.paths import EXPERIMENT_RESULTS_PATH

2024-05-06 08:25:35.114 | WARNING  | repsim.benchmark.paths:get_experiments_path:20 - No 'DATA_RESULTS_FOLDER' Env variable -- Defaulting to '<project_root>/experiments' .


In [3]:
path = "/root/similaritybench/experiments/results/nlp_aug_mnli_full.csv"
path = "/root/similaritybench/experiments/results/nlp_sc_mnli_full.csv"


cleaned_dfs = []
for path, setting, dataset in [
        ("/root/similaritybench/experiments/results/nlp_aug_sst2.csv", "aug", "sst2"),
        ("/root/similaritybench/experiments/results/nlp_mem_sst2.csv", "mem", "sst2"),
        ("/root/similaritybench/experiments/results/nlp_shortcut_sst2.csv", "sc", "sst2"),
        ("/root/similaritybench/experiments/results/nlp_shortcut_mnli.csv", "sc", "mnli"),
        ("/root/similaritybench/experiments/results/nlp_aug_mnli.csv", "aug", "mnli"),
        ("/root/similaritybench/experiments/results/nlp_mem_mnli.csv", "mem", "mnli"),
    ]:
    df = pd.read_csv(path)
    data = df.loc[2:].copy().reset_index(drop=True)
    data.columns = ["Similarity Measure", "Violation Rate", "AUPRC"]
    data["Architecture"] = "BERT-L"
    data.loc[:, "Violation Rate"] = data.loc[:, "Violation Rate"].astype(float)
    data.loc[:, "AUPRC"] = data.loc[:, "AUPRC"].astype(float)
    data = data.melt(
        id_vars=["Similarity Measure", "Architecture"],
        value_vars=["Violation Rate", "AUPRC"],
        var_name="Quality Metric",
        value_name="Score",
    )
    data["Setting"] = setting
    data["Dataset"] = dataset

    cleaned_dfs.append(data)

data = pd.concat(cleaned_dfs).reset_index(drop=True)
data.head()

,Similarity Measure,Architecture,Quality Metric,Score,Setting,Dataset
0,AlignedCosineSimilarity,BERT-L,Violation Rate,0.408722,aug,sst2
1,CKA,BERT-L,Violation Rate,0.342889,aug,sst2
2,ConcentricityDifference,BERT-L,Violation Rate,0.467389,aug,sst2
3,DistanceCorrelation,BERT-L,Violation Rate,0.345522,aug,sst2
4,Gulp,BERT-L,Violation Rate,0.531089,aug,sst2


In [7]:
for setting in data.Setting.unique():
    pivot = pd.pivot(data[data.Setting == setting], index="Similarity Measure", columns=["Quality Metric", "Dataset"], values="Score")
    print(pivot.to_latex())

\begin{tabular}{lllll}
\toprule
Quality Metric & Violation Rate & AUPRC & Violation Rate & AUPRC \\
Dataset & sst2 & sst2 & mnli & mnli \\
Similarity Measure &  &  &  &  \\
\midrule
AlignedCosineSimilarity & 0.408722 & 0.133663 & 0.262800 & 0.183733 \\
CKA & 0.342889 & 0.205222 & 0.179800 & 0.291237 \\
ConcentricityDifference & 0.467389 & 0.116144 & 0.425800 & 0.156790 \\
DistanceCorrelation & 0.345522 & 0.202842 & 0.184000 & 0.287191 \\
EigenspaceOverlapScore & NaN & NaN & 0.282200 & 0.156791 \\
Gulp & 0.531089 & 0.100955 & 0.317800 & 0.143141 \\
HardCorrelationMatch & 0.490944 & 0.104825 & 0.367600 & 0.125508 \\
JaccardSimilarity & 0.422944 & 0.120932 & 0.282800 & 0.223945 \\
MagnitudeDifference & 0.510133 & 0.106836 & 0.544400 & 0.102424 \\
OrthogonalAngularShapeMetricCentered & 0.391900 & 0.155239 & 0.226200 & 0.205825 \\
OrthogonalProcrustesCenteredAndNormalized & 0.391900 & 0.155229 & 0.226200 & 0.205825 \\
PWCCA & 0.494895 & NaN & 0.484669 & 0.115612 \\
PermutationProcrustes & 0

In [12]:
pivot = data.pivot_table(index="Similarity Measure", columns=["Quality Metric", "Setting", "Dataset"], values="Score")
pivot


Quality Metric                                AUPRC                      \
Setting                                         aug                 mem   
Dataset                                        mnli      sst2      sst2   
Similarity Measure                                                        
AlignedCosineSimilarity                    0.183733  0.133663  0.512842   
CKA                                        0.291237  0.205222  0.218001   
ConcentricityDifference                     0.15679  0.116144  0.737925   
DistanceCorrelation                        0.287191  0.202842  0.221519   
EigenspaceOverlapScore                     0.156791       NaN       NaN   
Gulp                                       0.143141  0.100955  0.149911   
HardCorrelationMatch                       0.125508  0.104825  0.202568   
JaccardSimilarity                          0.223945  0.120932  0.129887   
MagnitudeDifference                        0.102424  0.106836   0.15187   
OrthogonalAngularShapeMetricCentered       0.205825  0.155239  0.162086   
OrthogonalProcrustesCenteredAndNormalized  0.205825  0.155229  0.162086   
PWCCA                                      0.115612       NaN       NaN   
PermutationProcrustes                      0.093211  0.101906  0.232547   
ProcrustesSizeAndShapeDistance               0.1545  0.133479  0.448783   
RSA                                        0.299872  0.203099  0.154634   
RSMNormDifference                          0.090909  0.101124       NaN   
RankSimilarity                             0.216713  0.116057  0.127127   
SVCCA                                      0.241232  0.202352  0.256518   
SecondOrderCosineSimilarity                0.291765  0.132104  0.140627   
SoftCorrelationMatch                       0.141624  0.111305  0.226598   
UniformityDifference                       0.253374  0.107856  0.333816   

Quality Metric                                                Violation Rate  \
Setting                                          sc                      aug   
Dataset                                        mnli      sst2           mnli   
Similarity Measure                                                             
AlignedCosineSimilarity                    0.305176  0.242226         0.2628   
CKA                                         0.30916  0.125756         0.1798   
ConcentricityDifference                    0.193628  0.125556         0.4258   
DistanceCorrelation                         0.30846  0.130098          0.184   
EigenspaceOverlapScore                          NaN       NaN         0.2822   
Gulp                                       0.299563  0.122657         0.3178   
HardCorrelationMatch                       0.301422  0.110853         0.3676   
JaccardSimilarity                          0.301301   0.30186         0.2828   
MagnitudeDifference                        0.256078  0.152653         0.5444   
OrthogonalAngularShapeMetricCentered       0.317218  0.141282         0.2262   
OrthogonalProcrustesCenteredAndNormalized  0.317218  0.141284         0.2262   
PWCCA                                       0.29891       NaN       0.484669   
PermutationProcrustes                      0.286305  0.250116         0.5192   
ProcrustesSizeAndShapeDistance              0.26062  0.205246         0.3244   
RSA                                        0.304199   0.17344         0.1774   
RSMNormDifference                          0.090909       NaN            1.0   
RankSimilarity                             0.303617   0.30421         0.3034   
SVCCA                                      0.202846  0.125645          0.293   
SecondOrderCosineSimilarity                0.300228  0.281961         0.3482   
SoftCorrelationMatch                        0.30125  0.109047         0.3278   
UniformityDifference                       0.189129  0.246912         0.3678   

Quality Metric                                                          \
Setting                                          

In [16]:
styled = pd.io.formats.style.Styler(
    pivot,
    precision=3,
    caption="Ability to separate groups of BERT representations in different settings.",
)
styled

In [21]:
print(styled.to_latex(hrules=True, position="t", label="tab:nlp_groupsep", ))

\begin{table}[t]
\caption{Ability to separate groups of BERT representations in different settings.}
\label{tab:nlp_groupsep}
\begin{tabular}{lllllllllll}
\toprule
Quality Metric & \multicolumn{5}{r}{AUPRC} & \multicolumn{5}{r}{Violation Rate} \\
Setting & \multicolumn{2}{r}{aug} & mem & \multicolumn{2}{r}{sc} & \multicolumn{2}{r}{aug} & mem & \multicolumn{2}{r}{sc} \\
Dataset & mnli & sst2 & sst2 & mnli & sst2 & mnli & sst2 & sst2 & mnli & sst2 \\
Similarity Measure &  &  &  &  &  &  &  &  &  &  \\
\midrule
AlignedCosineSimilarity & 0.184 & 0.134 & 0.513 & 0.305 & 0.242 & 0.263 & 0.409 & 0.102 & 0.283 & 0.327 \\
CKA & 0.291 & 0.205 & 0.218 & 0.309 & 0.126 & 0.180 & 0.343 & 0.327 & 0.261 & 0.431 \\
ConcentricityDifference & 0.157 & 0.116 & 0.738 & 0.194 & 0.126 & 0.426 & 0.467 & 0.048 & 0.351 & 0.411 \\
DistanceCorrelation & 0.287 & 0.203 & 0.222 & 0.308 & 0.130 & 0.184 & 0.346 & 0.308 & 0.271 & 0.418 \\
EigenspaceOverlapScore & 0.157 & nan & nan & nan & nan & 0.282 & nan & nan & nan &